# Module 02: Branching and Merging

**Difficulty**: ⭐ Beginner

**Estimated Time**: 60-90 minutes

**Prerequisites**: 
- [Module 00: Setup and Introduction](00_setup_and_introduction.ipynb)
- [Module 01: Git Fundamentals](01_git_fundamentals.ipynb)

---

## Learning Objectives

By the end of this notebook, you will be able to:

1. Explain what branches are and why they're essential in Git
2. Create and delete branches using multiple Git commands
3. Switch between branches safely
4. Merge branches using fast-forward and 3-way merge strategies
5. Visualize branch structures and understand commit graphs
6. Apply branching best practices to your workflow

---

## 1. What Are Branches?

### The Concept

A **branch** is an independent line of development. Think of it as a parallel universe for your code where you can experiment without affecting the main codebase.

```
        main branch
          |
    A --- B --- C --- D
           \
            E --- F    feature branch
```

### Why Use Branches?

**Without branches**: You'd have to finish every feature completely before starting another, or risk breaking your working code.

**With branches**: You can:
- Develop features independently
- Fix bugs without disrupting ongoing work
- Experiment safely (delete branch if experiment fails)
- Collaborate without stepping on each other's toes
- Keep a stable main branch while working on new features

### Real-World Scenario

Imagine you're building a website:

- **main**: Production-ready code that's live
- **feature/login**: New login system (in progress)
- **feature/payment**: Payment integration (in progress)
- **bugfix/typo**: Quick fix for homepage typo

Each developer works on their branch, and changes are merged when ready!

---

## 2. Setting Up a Practice Repository

Let's create a repository to practice branching.

In [ ]:
import os

# Create practice directory
practice_dir = "../outputs/branching_practice"
os.makedirs(practice_dir, exist_ok=True)

print(f"Created directory: {practice_dir}")
print(f"Absolute path: {os.path.abspath(practice_dir)}")

In [ ]:
# Initialize repository
%cd {practice_dir}
!git init

In [ ]:
# Create initial file and commit
with open("app.py", "w") as f:
    f.write("""# Main Application
version = "1.0.0"

def main():
    print(f"Application v{version}")

if __name__ == "__main__":
    main()
""")

!git add app.py
!git commit -m "Initial commit: Add main application file"
print("Initial commit created!")

---

## 3. Viewing and Creating Branches

### List All Branches

In [ ]:
# List branches (the * shows current branch)
!git branch

**Notice**: By default, you're on the `main` branch (or `master` in older Git versions).

### Creating a New Branch (Method 1: git branch)

In [ ]:
# Create a new branch called 'feature/user-login'
!git branch feature/user-login

# List branches to see the new branch
!git branch

**Note**: `git branch <name>` creates a branch but doesn't switch to it. You're still on `main`!

### Creating and Switching (Method 2: git checkout)

In [ ]:
# Create and switch to a new branch in one command
!git checkout -b feature/dashboard

# Verify current branch
!git branch

**Syntax**: `git checkout -b <branch-name>` = create + switch

### Creating and Switching (Method 3: git switch - Modern)

In [ ]:
# Git 2.23+ introduced 'git switch' for better clarity
!git switch -c feature/settings

# Verify
!git branch

**Modern best practice**: Use `git switch` for changing branches, `git checkout` for other operations.

```bash
# Old way (still works)
git checkout -b new-branch
git checkout existing-branch

# New way (clearer)
git switch -c new-branch      # Create and switch
git switch existing-branch    # Just switch
```

---

## 4. Switching Between Branches

### Switch Back to Main Branch

In [ ]:
# Switch to main branch
!git switch main

# Or use the older command
# !git checkout main

# Verify
!git branch

### What Happens When You Switch?

When you switch branches, Git:
1. Updates files in your working directory to match the branch
2. Updates HEAD to point to the new branch
3. Updates the staging area

**Important**: Git won't let you switch if you have uncommitted changes that would be lost. Commit or stash them first!

---

## 5. Working on a Branch

Let's make changes on a feature branch and see how it differs from main.

### Create and Work on Feature Branch

In [ ]:
# Create new feature branch
!git switch -c feature/add-authentication

# Add authentication code
with open("auth.py", "w") as f:
    f.write("""# Authentication Module

def login(username, password):
    # Placeholder authentication logic
    if username and password:
        return True
    return False

def logout():
    print("User logged out successfully")
""")

# Commit the changes
!git add auth.py
!git commit -m "Add authentication module with login/logout functions"

print("Authentication feature added!")

In [ ]:
# Check files in current branch
!ls -la

### Switch Back to Main and Compare

In [ ]:
# Switch back to main
!git switch main

# Check files - auth.py should be gone!
!ls -la

**Notice**: The `auth.py` file disappeared! It only exists on the `feature/add-authentication` branch.

### View Commit History Differences

In [ ]:
# View main branch history
print("=== MAIN BRANCH ===")
!git log --oneline

print("\n=== FEATURE BRANCH ===")
!git log --oneline feature/add-authentication

---

## 6. Visualizing Branches

### Text-Based Branch Visualization

In [ ]:
# Show branch graph with all branches
!git log --oneline --graph --all --decorate

**Reading the graph**:
- `*` = commit
- `|` = branch line
- `/` and `\` = branch/merge points
- Different colors = different branches

### Create a More Complex Branch Structure

In [ ]:
# Make a commit on main
!git switch main
with open("README.md", "w") as f:
    f.write("# My Application\n\nMain project readme")
!git add README.md
!git commit -m "Add README on main branch"

# Make another commit on feature branch
!git switch feature/add-authentication
with open("auth.py", "a") as f:
    f.write("\ndef validate_password(password):\n    return len(password) >= 8\n")
!git commit -am "Add password validation function"

# View the diverged branches
!git log --oneline --graph --all --decorate

**Notice**: The branches have diverged! They share common history but have different commits.

---

## 7. Merging Branches

Once a feature is complete, you merge it back into the main branch.

### Merge Strategies

Git uses two main merge strategies:

#### 1. Fast-Forward Merge
When main hasn't changed since the branch was created:

```
Before:                    After:
  main                       main
    |                          |
    A --- B --- C              A --- B --- C
           \
            D --- E  feature
```

#### 2. Three-Way Merge
When both branches have new commits:

```
Before:                    After:
  main                       main
    |                          |
    A --- B --- C              A --- B --- C --- M
           \                            \     /
            D --- E  feature             D - E
```

### Performing a Fast-Forward Merge

In [ ]:
# Create a simple branch for fast-forward demo
!git switch main
!git switch -c feature/quick-fix

# Make a change
with open("app.py", "w") as f:
    f.write("""# Main Application
version = "1.0.1"  # Bug fix version

def main():
    print(f"Application v{version}")

if __name__ == "__main__":
    main()
""")

!git commit -am "Bump version to 1.0.1"

# Switch to main and merge
!git switch main
!git merge feature/quick-fix

**Notice**: The output says "Fast-forward" - Git simply moved the main pointer forward!

### Performing a Three-Way Merge

In [ ]:
# We already have diverged branches, let's merge authentication
!git switch main
!git merge feature/add-authentication -m "Merge authentication feature into main"

**Notice**: Git created a merge commit to combine the two branches!

### View the Merge Result

In [ ]:
# See the full graph
!git log --oneline --graph --all

In [ ]:
# Verify that auth.py is now on main
!ls -la

---

## 8. Deleting Branches

After merging a feature, you typically delete the feature branch.

### Safe Deletion (Merged Branches Only)

In [ ]:
# Delete merged branches
!git branch -d feature/quick-fix
!git branch -d feature/add-authentication

# View remaining branches
!git branch

**Note**: `-d` (lowercase) only deletes branches that have been merged. This is safe!

### Force Deletion (Unmerged Branches)

In [ ]:
# Try to delete an unmerged branch
!git branch -d feature/dashboard

Git warns us! The branch has commits that aren't merged.

To force delete:

In [ ]:
# Force delete with -D (uppercase)
!git branch -D feature/dashboard
!git branch -D feature/settings
!git branch -D feature/user-login

# View final branches
!git branch

**Warning**: `-D` (uppercase) force deletes branches even if unmerged. You'll lose those commits!

---

## 9. Branch Naming Conventions

### Common Patterns

**Feature Branches**:
```
feature/user-authentication
feature/payment-integration
feat/new-dashboard
```

**Bug Fixes**:
```
bugfix/login-error
fix/memory-leak
hotfix/critical-security-issue
```

**Releases**:
```
release/v1.2.0
release/2024-01
```

**Experiments**:
```
experiment/new-algorithm
poc/blockchain-integration
```

### Best Practices

**Do**:
- Use lowercase with hyphens: `feature/user-login`
- Be descriptive: `fix/navbar-responsive-issue`
- Use prefixes: `feature/`, `bugfix/`, `hotfix/`
- Keep names concise but clear

**Don't**:
- Use spaces: ❌ `feature/new stuff`
- Be vague: ❌ `fix`, `test`, `temp`
- Use special characters: ❌ `feature/user@login`
- Start with a hyphen: ❌ `-feature`

---

## 10. Common Mistakes and How to Avoid Them

### Mistake 1: Forgetting Which Branch You're On

In [ ]:
# Always check current branch before committing
!git branch
# Or use status
!git status

**Solution**: Configure your terminal prompt to show current branch!

### Mistake 2: Switching with Uncommitted Changes

In [ ]:
# Create a change
with open("test.txt", "w") as f:
    f.write("Uncommitted change")

# Try to switch (this might fail)
!git switch -c new-branch

**Solutions**:
1. Commit the changes first
2. Use `git stash` to temporarily save changes (we'll cover this later)
3. Discard changes with `git restore <file>`

### Mistake 3: Merging the Wrong Direction

In [ ]:
# Clean up uncommitted changes first
!git restore test.txt
!rm -f test.txt 2>nul || del test.txt 2>nul

# CORRECT: Switch to destination, merge from source
# !git switch main
# !git merge feature/my-feature

# WRONG: Merging main into feature accidentally
# !git switch feature/my-feature
# !git merge main  # This merges main INTO feature!

print("Remember: git switch DESTINATION, then git merge SOURCE")

---

## 11. Exercises

### Exercise 1: Basic Branching Workflow

1. Create a new Git repository
2. Add an initial file and commit it
3. Create a branch called `feature/new-feature`
4. Switch to the new branch
5. Add a new file and commit it
6. Switch back to main
7. Merge the feature branch into main
8. Delete the feature branch

In [ ]:
# Your code here for Exercise 1


### Exercise 2: Multiple Feature Branches

1. Create two feature branches: `feature/header` and `feature/footer`
2. On `feature/header`, create a file called `header.html`
3. On `feature/footer`, create a file called `footer.html`
4. Merge both branches into main
5. Visualize the commit graph with `git log --graph`
6. Delete both feature branches

In [ ]:
# Your code here for Exercise 2


### Exercise 3: Fast-Forward vs Three-Way Merge

1. Create a feature branch from main
2. Make a commit on the feature branch
3. WITHOUT switching branches, make a commit on main (hint: use a different terminal or commit directly on GitHub)
4. Merge the feature branch - it should be a three-way merge
5. View the merge commit in the log

In [ ]:
# Your code here for Exercise 3


---

## 12. Best Practices Summary

### Branch Management

1. **Create branches frequently**: One per feature/bugfix
2. **Keep branches focused**: One purpose per branch
3. **Use descriptive names**: `feature/user-authentication` not `stuff`
4. **Delete merged branches**: Keep your branch list clean
5. **Merge regularly**: Don't let branches drift too far from main

### Merging Strategy

1. **Always switch to destination first**: `git switch main` then `git merge feature`
2. **Check status before merging**: `git status` should be clean
3. **Test before merging to main**: Ensure features work
4. **Write clear merge commit messages**: Explain what the merge adds
5. **Review changes**: Use `git diff main..feature` before merging

### When to Branch

**Always branch for**:
- New features
- Bug fixes
- Experiments
- Major refactoring

**Don't branch for**:
- Tiny typo fixes (commit directly to main)
- Documentation updates (unless major rewrite)

---

## 13. Knowledge Check

Before moving on, make sure you can answer these questions:

1. What is a branch in Git?
2. What's the difference between `git branch` and `git switch -c`?
3. How do you see which branch you're currently on?
4. What's the difference between a fast-forward merge and a three-way merge?
5. How do you delete a branch? What's the difference between `-d` and `-D`?
6. What happens to your files when you switch branches?
7. Why is it important to commit changes before switching branches?

### Practical Checklist

Can you:
- [ ] Create a new branch
- [ ] Switch between branches
- [ ] Make commits on a branch
- [ ] Merge a branch into another branch
- [ ] Delete a branch after merging
- [ ] Visualize branch history with `git log --graph`
- [ ] Explain when to use branches

---

## 14. Quick Reference: Branch Commands

```bash
# Creating Branches
git branch <branch-name>           # Create branch (don't switch)
git switch -c <branch-name>        # Create and switch (modern)
git checkout -b <branch-name>      # Create and switch (traditional)

# Switching Branches
git switch <branch-name>           # Switch to branch (modern)
git checkout <branch-name>         # Switch to branch (traditional)

# Viewing Branches
git branch                         # List local branches
git branch -a                      # List all branches (including remote)
git branch -v                      # List with last commit

# Merging
git merge <branch-name>            # Merge branch into current branch
git merge --no-ff <branch-name>    # Force merge commit (no fast-forward)
git merge --abort                  # Cancel merge if there's a conflict

# Deleting Branches
git branch -d <branch-name>        # Delete merged branch (safe)
git branch -D <branch-name>        # Force delete branch (dangerous)

# Viewing Branch History
git log --graph --oneline --all    # Visual branch history
git show-branch                    # Show branch relationships
```

---

## 15. Summary

In this module, you learned:

- ✅ What branches are and why they're fundamental to Git workflows
- ✅ How to create branches using `git branch` and `git switch -c`
- ✅ How to switch between branches safely
- ✅ The difference between fast-forward and three-way merges
- ✅ How to merge branches and handle merge commits
- ✅ How to visualize branch structures with `git log --graph`
- ✅ Branch naming conventions and best practices
- ✅ How to delete branches properly

Branches are the foundation of collaborative Git workflows. Master them, and you're well on your way to becoming a Git expert!

---

## 16. Next Steps

**Next Module**: [Module 03: Remote Repositories and GitHub](03_remote_repositories_and_github.ipynb)

In the next module, you'll learn:
- How to connect local repositories to GitHub
- Pushing and pulling code between local and remote repositories
- Cloning repositories from GitHub
- Managing remote branches
- Understanding origin and upstream
- Authentication methods (SSH vs HTTPS)

Take a break and practice branching before moving on!

---

**Great work! See you in Module 03!**